# Learning of Process Representations Using Recurrent Neural Networks

In [11]:
# added more imports like tqdm, evaluation
import numpy as np
import pandas as pd

from replearn.eventlog import EventLog

from replearn.embedding_predict import EmbeddingPredict
from replearn.autoencoder import AutoencoderRepresentation
from replearn.doc2vec import Doc2VecRepresentation

from replearn.clustering import Clustering
from replearn.evaluation import Evaluation

from tqdm import tqdm
import os
import re

### General Parameters

In [12]:
event_log_path = '../logs/'
case_attributes = None # auto-detect attributes
event_attributes = ['concept:name', 'user'] # use activity name and user
true_cluster_label = 'cluster'

n_epochs = 25         #[10, 25]
n_batch_size = 64
n_clusters = 5
vector_size = 32      #[2, 3, 4, 8, 16, 32, 64, 128]

clustering_method = "agglomerative" # ["k_means", "agglomerative"]

## Load event log

In [13]:
# noise event logs
event_logs = {}
for i in range(11):
    noise = i / 10.0
    event_logs[noise] = []

# prepare all event log files
pattern = r'-(\d+\.\d+)'
for file in tqdm(os.listdir(event_log_path), unit='eventlog'):
    if os.path.isfile(os.path.join(event_log_path, file)):
        match = re.search(pattern, file)
        if match:
            noise = float(match.group(1))
            event_logs[noise].append(file)
            
print(event_logs.keys())
print("First three:", event_logs[0.0][0:3])

100%|██████████████████████████████████████████████████████████████████████████| 11/11 [00:00<00:00, 3674.23eventlog/s]

dict_keys([0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
First three: ['small_500_10_20_5_1_1-0.0-1.json.gz']


## Representation Learning

In [14]:
# hyperparameters
n_epochs = 10
n_batch_size = 64
n_clusters = 5

vector_size = 32

### GRU

In [15]:
# init and train GRU - added noise filtering, loop over all logs, metrics like f-score, statistics to create graph on
# statistics saved in google docs excel file in README to utilize further for presentation

#GRUClust
results_gruclust = {}
for i in range(11):
    noise = i / 10.0
    results_gruclust[noise] = []

for noise in tqdm(event_logs.keys(), unit="noise_level"):    
    for file in tqdm(event_logs[noise], unit='event_log'):
        # load file
        event_log = EventLog(file, case_attributes=case_attributes, event_attributes=event_attributes, true_cluster_label=true_cluster_label)
        event_log.load(event_log_path + file, False)
        event_log.preprocess()
        
        # get sequences from event log as one-hot feature vector
        sequences = event_log.event_attributes_flat_onehot_features_2d

        predictor = EmbeddingPredict(event_log)
        predictor.build_model(embedding_dim=vector_size, gru_dim=vector_size, rnn='gru')
        predictor.fit(epochs=n_epochs, batch_size=n_batch_size, verbose=True)   
        
        # infer the vector from the model
        pred_model, feature_vector, embedding_vector = predictor.predict()    
        
        cluster_analysis = Clustering(event_log)
        cluster_analysis.cluster(feature_vector, clustering_method, n_clusters, 'cosine')

        cluster_result = cluster_analysis.evaluate()
        evaluation = Evaluation(event_log)
        (fitness, precision, simplicity) = evaluation.evaluate_clusters(n_clusters, cluster_analysis.pred_labels) # Heuristics Miner + 2. Metric
        f_score = 2 * (fitness * precision) / (fitness + precision) # idea: fitness <=> recall & precision <=> precision
        
        results_gruclust[noise].append({"f1_bcubed":cluster_result[2], "f_score":f_score, "fitness": fitness, "precision":precision, "simplicity":simplicity})

  0%|                                                                                     | 0/1 [00:00<?, ?event_log/s]

Epoch 1/10
8/8 [==============================] - 13s 33ms/step - loss: 21.0454 - dense_loss: 1.9218 - dense_1_loss: 1.9704 - dense_2_loss: 2.2668 - dense_3_loss: 1.9885 - dense_4_loss: 2.4064 - dense_5_loss: 2.0870 - dense_6_loss: 2.1081 - dense_7_loss: 1.9702 - dense_8_loss: 2.0483 - dense_9_loss: 2.2780 - dense_accuracy: 0.2500 - dense_1_accuracy: 0.2080 - dense_2_accuracy: 0.1560 - dense_3_accuracy: 0.2500 - dense_4_accuracy: 0.1720 - dense_5_accuracy: 0.2040 - dense_6_accuracy: 0.2700 - dense_7_accuracy: 0.1840 - dense_8_accuracy: 0.2060 - dense_9_accuracy: 0.1140
Epoch 2/10
8/8 [==============================] - 0s 24ms/step - loss: 16.3830 - dense_loss: 1.3598 - dense_1_loss: 1.5840 - dense_2_loss: 1.7275 - dense_3_loss: 1.7085 - dense_4_loss: 1.8213 - dense_5_loss: 1.5714 - dense_6_loss: 1.5275 - dense_7_loss: 1.6610 - dense_8_loss: 1.5327 - dense_9_loss: 1.8893 - dense_accuracy: 0.4660 - dense_1_accuracy: 0.3420 - dense_2_accuracy: 0.2600 - dense_3_accuracy: 0.2520 - dense_4_a


  0%|                                                                                     | 0/1 [00:00<?, ?event_log/s]

Epoch 1/10
8/8 [==============================] - 14s 26ms/step - loss: 22.4676 - dense_10_loss: 2.2082 - dense_11_loss: 2.1416 - dense_12_loss: 2.4315 - dense_13_loss: 2.0254 - dense_14_loss: 2.6141 - dense_15_loss: 2.2236 - dense_16_loss: 2.2311 - dense_17_loss: 2.1939 - dense_18_loss: 2.1098 - dense_19_loss: 2.2883 - dense_10_accuracy: 0.1380 - dense_11_accuracy: 0.1680 - dense_12_accuracy: 0.1120 - dense_13_accuracy: 0.1740 - dense_14_accuracy: 0.1060 - dense_15_accuracy: 0.1800 - dense_16_accuracy: 0.2000 - dense_17_accuracy: 0.1240 - dense_18_accuracy: 0.2000 - dense_19_accuracy: 0.1440
Epoch 2/10
8/8 [==============================] - 0s 29ms/step - loss: 18.9534 - dense_10_loss: 1.7841 - dense_11_loss: 1.9165 - dense_12_loss: 2.0196 - dense_13_loss: 1.8186 - dense_14_loss: 2.1667 - dense_15_loss: 1.8779 - dense_16_loss: 1.7695 - dense_17_loss: 1.8387 - dense_18_loss: 1.7274 - dense_19_loss: 2.0345 - dense_10_accuracy: 0.2820 - dense_11_accuracy: 0.2520 - dense_12_accuracy: 0.19


  0%|                                                                                     | 0/1 [00:00<?, ?event_log/s]

...


  0%|                                                                                     | 0/1 [00:00<?, ?event_log/s]

Epoch 1/10
8/8 [==============================] - 14s 35ms/step - loss: 24.6597 - dense_90_loss: 2.3960 - dense_91_loss: 2.3927 - dense_92_loss: 2.5225 - dense_93_loss: 2.3198 - dense_94_loss: 2.8960 - dense_95_loss: 2.4869 - dense_96_loss: 2.5158 - dense_97_loss: 2.4105 - dense_98_loss: 2.2232 - dense_99_loss: 2.4963 - dense_90_accuracy: 0.2160 - dense_91_accuracy: 0.1260 - dense_92_accuracy: 0.1480 - dense_93_accuracy: 0.1180 - dense_94_accuracy: 0.0940 - dense_95_accuracy: 0.1340 - dense_96_accuracy: 0.1840 - dense_97_accuracy: 0.1700 - dense_98_accuracy: 0.1920 - dense_99_accuracy: 0.1700
Epoch 2/10
8/8 [==============================] - 0s 30ms/step - loss: 22.0030 - dense_90_loss: 2.0799 - dense_91_loss: 2.1430 - dense_92_loss: 2.3351 - dense_93_loss: 2.0691 - dense_94_loss: 2.5397 - dense_95_loss: 2.2379 - dense_96_loss: 2.2162 - dense_97_loss: 2.1098 - dense_98_loss: 2.0316 - dense_99_loss: 2.2406 - dense_90_accuracy: 0.3040 - dense_91_accuracy: 0.2300 - dense_92_accuracy: 0.17


  0%|                                                                                     | 0/1 [00:00<?, ?event_log/s]

Epoch 1/10
8/8 [==============================] - 13s 26ms/step - loss: 24.8316 - dense_100_loss: 2.4796 - dense_101_loss: 2.3872 - dense_102_loss: 2.5296 - dense_103_loss: 2.3732 - dense_104_loss: 2.8973 - dense_105_loss: 2.4711 - dense_106_loss: 2.5740 - dense_107_loss: 2.3165 - dense_108_loss: 2.3243 - dense_109_loss: 2.4786 - dense_100_accuracy: 0.1060 - dense_101_accuracy: 0.1300 - dense_102_accuracy: 0.0960 - dense_103_accuracy: 0.1540 - dense_104_accuracy: 0.0600 - dense_105_accuracy: 0.0860 - dense_106_accuracy: 0.0780 - dense_107_accuracy: 0.2280 - dense_108_accuracy: 0.1140 - dense_109_accuracy: 0.0980
Epoch 2/10
8/8 [==============================] - 0s 26ms/step - loss: 22.4358 - dense_100_loss: 2.1274 - dense_101_loss: 2.1403 - dense_102_loss: 2.3394 - dense_103_loss: 2.0994 - dense_104_loss: 2.6368 - dense_105_loss: 2.2235 - dense_106_loss: 2.3089 - dense_107_loss: 2.0934 - dense_108_loss: 2.1469 - dense_109_loss: 2.3198 - dense_100_accuracy: 0.2040 - dense_101_accuracy: 


100%|█████████████████████████████████████████████████████████████████████████| 11/11 [03:37<00:00, 19.77s/noise_level]


In [18]:
# Statistics GRU

# complete
complete = []
for i in range(11):
    complete += results_gruclust[i/10]
complete = pd.DataFrame.from_dict(complete)
print(complete.describe())

print("---")

# noise
for i in range(11):
    print(f"noise: {i/10}")
    print(pd.DataFrame.from_dict(results_gruclust[i/10]).describe())
    print("---")

       f1_bcubed    f_score    fitness  precision  simplicity
count  11.000000  11.000000  11.000000  11.000000   11.000000
mean    0.924485   0.648969   0.707216   0.604185    0.644677
std     0.037669   0.090937   0.084260   0.108425    0.084689
min     0.854812   0.488746   0.532383   0.451721    0.484000
25%     0.906433   0.580065   0.665202   0.488733    0.608873
50%     0.919701   0.667821   0.723871   0.634864    0.636857
75%     0.949292   0.720178   0.767371   0.699338    0.698663
max     0.978967   0.762060   0.806000   0.725518    0.806000
---
noise: 0.0
       f1_bcubed   f_score  fitness  precision  simplicity
count   1.000000  1.000000    1.000    1.00000       1.000
mean    0.854812  0.611144    0.806    0.49216       0.806
std          NaN       NaN      NaN        NaN         NaN
min     0.854812  0.611144    0.806    0.49216       0.806
25%     0.854812  0.611144    0.806    0.49216       0.806
50%     0.854812  0.611144    0.806    0.49216       0.806
75%     0.8548

### LSTM

In [19]:
# init and train LSTM - added noise filtering, loop over all logs, metrics like f-score, statistics to create graph on
# statistics saved in google docs excel file in README to utilize further for presentation

results_lstmclust= {}
for i in range(11):
    noise = i / 10.0
    results_lstmclust[noise] = []

for noise in tqdm(event_logs.keys(), unit="noise_level"):    
    for file in tqdm(event_logs[noise], unit='event_log'):
        # load file
        event_log = EventLog(file, case_attributes=case_attributes, event_attributes=event_attributes, true_cluster_label=true_cluster_label)
        event_log.load(event_log_path + file, False)
        event_log.preprocess()
        
        # get sequences from event log as one-hot feature vector
        sequences = event_log.event_attributes_flat_onehot_features_2d

        predictor = EmbeddingPredict(event_log)
        predictor.build_model(embedding_dim=vector_size, gru_dim=vector_size, rnn='LSTM')
        predictor.fit(epochs=n_epochs, batch_size=n_batch_size, verbose=True)   
        
        # infer the vector from the model
        pred_model, feature_vector, embedding_vector = predictor.predict()    
        
        cluster_analysis = Clustering(event_log)
        cluster_analysis.cluster(feature_vector, clustering_method, n_clusters, 'cosine')

        cluster_result = cluster_analysis.evaluate()
        evaluation = Evaluation(event_log)
        (fitness, precision, simplicity) = evaluation.evaluate_clusters(n_clusters, cluster_analysis.pred_labels) # Heuristics Miner + 2. Metric
        f_score = 2 * (fitness * precision) / (fitness + precision) # idea: fitness <=> recall & precision <=> precision
        
        results_lstmclust[noise].append({"f1_bcubed":cluster_result[2], "f_score":f_score, "fitness": fitness, "precision":precision, "simplicity":simplicity})

  0%|                                                                                     | 0/1 [00:00<?, ?event_log/s]

Epoch 1/10
8/8 [==============================] - 13s 22ms/step - loss: 21.3502 - dense_220_loss: 1.9601 - dense_221_loss: 1.9945 - dense_222_loss: 2.2975 - dense_223_loss: 2.0276 - dense_224_loss: 2.4123 - dense_225_loss: 2.1441 - dense_226_loss: 2.1594 - dense_227_loss: 1.9818 - dense_228_loss: 2.1256 - dense_229_loss: 2.2473 - dense_220_accuracy: 0.3160 - dense_221_accuracy: 0.2400 - dense_222_accuracy: 0.1460 - dense_223_accuracy: 0.1340 - dense_224_accuracy: 0.1200 - dense_225_accuracy: 0.1200 - dense_226_accuracy: 0.2140 - dense_227_accuracy: 0.2260 - dense_228_accuracy: 0.2620 - dense_229_accuracy: 0.1880
Epoch 2/10
8/8 [==============================] - 0s 23ms/step - loss: 17.2254 - dense_220_loss: 1.4912 - dense_221_loss: 1.6309 - dense_222_loss: 1.8318 - dense_223_loss: 1.7294 - dense_224_loss: 1.9318 - dense_225_loss: 1.7436 - dense_226_loss: 1.6152 - dense_227_loss: 1.6830 - dense_228_loss: 1.5907 - dense_229_loss: 1.9778 - dense_220_accuracy: 0.4640 - dense_221_accuracy: 


  0%|                                                                                     | 0/1 [00:00<?, ?event_log/s]

Epoch 1/10
8/8 [==============================] - 15s 32ms/step - loss: 22.3458 - dense_230_loss: 2.1732 - dense_231_loss: 2.1166 - dense_232_loss: 2.4130 - dense_233_loss: 2.0102 - dense_234_loss: 2.6334 - dense_235_loss: 2.2679 - dense_236_loss: 2.2006 - dense_237_loss: 2.0856 - dense_238_loss: 2.1582 - dense_239_loss: 2.2871 - dense_230_accuracy: 0.1860 - dense_231_accuracy: 0.1640 - dense_232_accuracy: 0.1760 - dense_233_accuracy: 0.2600 - dense_234_accuracy: 0.1040 - dense_235_accuracy: 0.1220 - dense_236_accuracy: 0.2280 - dense_237_accuracy: 0.1820 - dense_238_accuracy: 0.1140 - dense_239_accuracy: 0.1320
Epoch 2/10
8/8 [==============================] - 0s 28ms/step - loss: 18.9918 - dense_230_loss: 1.8185 - dense_231_loss: 1.7916 - dense_232_loss: 2.0479 - dense_233_loss: 1.7980 - dense_234_loss: 2.2116 - dense_235_loss: 1.9497 - dense_236_loss: 1.7786 - dense_237_loss: 1.7824 - dense_238_loss: 1.7776 - dense_239_loss: 2.0359 - dense_230_accuracy: 0.3200 - dense_231_accuracy: 


  0%|                                                                                     | 0/1 [00:00<?, ?event_log/s]

...


  0%|                                                                                     | 0/1 [00:00<?, ?event_log/s]

Epoch 1/10
8/8 [==============================] - 13s 31ms/step - loss: 24.5062 - dense_310_loss: 2.3943 - dense_311_loss: 2.3461 - dense_312_loss: 2.5243 - dense_313_loss: 2.3122 - dense_314_loss: 2.8709 - dense_315_loss: 2.4590 - dense_316_loss: 2.5210 - dense_317_loss: 2.3581 - dense_318_loss: 2.2378 - dense_319_loss: 2.4825 - dense_310_accuracy: 0.1720 - dense_311_accuracy: 0.0980 - dense_312_accuracy: 0.1020 - dense_313_accuracy: 0.2400 - dense_314_accuracy: 0.0820 - dense_315_accuracy: 0.1520 - dense_316_accuracy: 0.1740 - dense_317_accuracy: 0.2020 - dense_318_accuracy: 0.2100 - dense_319_accuracy: 0.1640
Epoch 2/10
8/8 [==============================] - 0s 26ms/step - loss: 21.7147 - dense_310_loss: 2.0477 - dense_311_loss: 2.1220 - dense_312_loss: 2.3165 - dense_313_loss: 2.0040 - dense_314_loss: 2.4420 - dense_315_loss: 2.2225 - dense_316_loss: 2.1916 - dense_317_loss: 2.0803 - dense_318_loss: 2.0121 - dense_319_loss: 2.2761 - dense_310_accuracy: 0.2960 - dense_311_accuracy: 


  0%|                                                                                     | 0/1 [00:00<?, ?event_log/s]

Epoch 1/10
8/8 [==============================] - 14s 34ms/step - loss: 24.6677 - dense_320_loss: 2.4377 - dense_321_loss: 2.3988 - dense_322_loss: 2.5210 - dense_323_loss: 2.3348 - dense_324_loss: 2.9082 - dense_325_loss: 2.4580 - dense_326_loss: 2.5414 - dense_327_loss: 2.3712 - dense_328_loss: 2.2374 - dense_329_loss: 2.4590 - dense_320_accuracy: 0.1300 - dense_321_accuracy: 0.1320 - dense_322_accuracy: 0.1040 - dense_323_accuracy: 0.1480 - dense_324_accuracy: 0.0960 - dense_325_accuracy: 0.1320 - dense_326_accuracy: 0.0840 - dense_327_accuracy: 0.2440 - dense_328_accuracy: 0.1980 - dense_329_accuracy: 0.1360
Epoch 2/10
8/8 [==============================] - 0s 34ms/step - loss: 21.7745 - dense_320_loss: 1.9834 - dense_321_loss: 2.0894 - dense_322_loss: 2.2709 - dense_323_loss: 2.0973 - dense_324_loss: 2.6242 - dense_325_loss: 2.1760 - dense_326_loss: 2.2163 - dense_327_loss: 2.0759 - dense_328_loss: 2.0240 - dense_329_loss: 2.2170 - dense_320_accuracy: 0.3120 - dense_321_accuracy: 


100%|█████████████████████████████████████████████████████████████████████████| 11/11 [03:44<00:00, 20.43s/noise_level]


In [ ]:
# Statistics LSTM

# complete
complete = []
for i in range(11):
    complete += results_lstmclust[i/10]
complete = pd.DataFrame.from_dict(complete)
print(complete.describe())

print("---")

# noise
for i in range(11):
    print(f"noise: {i/10}")
    print(pd.DataFrame.from_dict(results_lstmclust[i/10]).describe())
    print("---")